<a href="https://colab.research.google.com/github/iraidaantropova/data-processing-method/blob/main/%D0%9F%D0%A07_Selenium_%D0%A1%D0%B8%D0%BD%D0%B8%D1%86%D0%B0%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Вариант I

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных 
(от кого, дата отправки, тема письма, текст письма полный)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import  MongoClient
import re
import time

chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://mail.ru/login') #захожу на сайт
print(driver.title)
assert "No results found." not in driver.page_source

auth_form = driver.find_element_by_id('auth-form')
iframe = auth_form.find_element_by_tag_name('iframe')
driver.switch_to.frame(iframe)

username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
username.send_keys("student.geek@bk.ru", Keys.ENTER)
password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'password')))
driver.implicitly_wait(1)
password.send_keys("83u-Qzq-bcg-Nw9", Keys.ENTER)

driver.switch_to.default_content()
driver.implicitly_wait(10)

# Считаю сколько писем в ящике
inbox_element = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME,'nav__item_active')))
title = inbox_element.get_attribute('title')
regex = r"Входящие, (\d*) "
count_emails = int(re.search(regex, title).group(1))
print(f"Всего писем: {count_emails}")

# Собираю список ссылок на письма
urls_marker = WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.CLASS_NAME,'js-letter-list-item')))
url_list = driver.find_elements_by_class_name('js-letter-list-item')
url_set = set()

for a in url_list:
    url_set.add(a.get_attribute('href'))

while len(url_set) != count_emails:
    actions = ActionChains(driver)
    actions.move_to_element(url_list[-1])
    actions.perform()
    time.sleep(1)
    url_list = driver.find_elements_by_class_name('js-letter-list-item')
    for a in url_list:
        url_set.add(a.get_attribute('href'))
    print(f"Собрано URL'ов: {len(url_set)}")

# Открываем каждое письмо и смотрим содержимое
emails = []
for a in url_set:
    driver.get(a)
    letter_author_wrapper = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'letter__author'))
    )
    email = {
        'letter_author': letter_author_wrapper.find_element_by_class_name('letter-contact').get_attribute('title'),
        'letter_date': letter_author_wrapper.find_element_by_class_name('letter__date').text,
        'letter_title': driver.find_element_by_class_name('thread__subject-line').text,
        'letter_body': driver.find_element_by_class_name('letter-body').text
    }
    emails.append(email)
    print(f"Обработана ссылка: {a}")

# Сохраняем в Базу данных
client = MongoClient('127.0.0.1', 27017)
db = client['emails']
db.inbox.insert_many(emails)
